In [1]:
import pandas as pd
import re


# 01 verify every quote link is in csv for download


This verification was done in the file `03_verify_index_links.ipynb` and the results show that _all the links in the index are in the csv_ file for download.


# 02 Verify every link has been downloaded and every file exists


The file with the link are `acm_site.csv`, `missionary.csv` and `stdhndbk.csv`


In [3]:
# get the files "acm_site.csv" and "missionary.csv" and merge them into a single dataframe
df_exist = pd.read_csv("../data/data_test_sep_27/all_links.csv")

# sort by filename
df_exist = df_exist.sort_values(by="filename")

df_exist.head(10)


,URL,Section,Subsection,Title,filename
494,https://www.byupathway.edu/policies/handbook/6...,['6. Student Records'],['Missing'],['6.4 Pathway Certificates'],105adb7a
311,https://missionaries.prod.byu-pathway.psdops.c...,"['Increasing Awareness', 'Shepherding / Minist...","['Increasing Awareness of PathwayConnect', 'Mi...","['Shepherd with Charity', 'Shepherd with Chari...",110f0051
180,https://missionaries.prod.byu-pathway.psdops.c...,['PathwayConnect (PC)'],['PathwayConnect General Information'],['How Students Repeat Courses'],12180468
288,https://missionaries.prod.byu-pathway.psdops.c...,['Missionary Software & Uses'],['Power BI'],['PowerBI'],123bdf7c
329,https://missionaries.prod.byu-pathway.psdops.c...,['Missionary Software & Uses'],['PATH for Area Coordination Missionaries (ACM...,['Transferring a Student to a Different Program'],12a1fef2
67,https://missionaries.prod.byu-pathway.psdops.c...,"['Calendars', 'Calendars']","['Calendar', 'EnglishConnect 3 Academic Calend...","['EC3 Auto-Drop', 'Auto-Drop for EC3']",130b6f2b
559,https://studentsupportkb.byupathway.org/knowle...,NaN,NaN,NaN,1376e9a9
333,https://missionaries.prod.byu-pathway.psdops.c...,"['Missionary Software & Uses', 'PATH']",['PATH for Area Coordination Missionaries (ACM...,"['Understanding the Groups Page EC3', 'Underst...",13c81944
522,https://www.englishconnect.org/,['English Connect 1&2'],['EnglishConnect Home'],['General Information'],141279b1
216,https://missionaries.prod.byu-pathway.psdops.c...,['Block'],['Missing'],['International Area Transitioning to Block a ...,143661b8


In [4]:
# read the names of the files in the directory "data/data_16_09_24/crawl/html" and "data/data_16_09_24/crawl/pdf"
import os

html_files = os.listdir("../data/data_test_sep_27/crawl/html")
pdf_files = os.listdir("../data/data_test_sep_27/crawl/pdf")

# ignore txt files that exist in md and txt with the same name in the "../data/data_16_09_24/crawl/pdf"

all_files = html_files + pdf_files

print("HTML files:", len(html_files))
print("PDF files:", len(pdf_files))
print("\n")
print("TOtal files:", len(html_files) + len(pdf_files), "plus 1 files as image.")


HTML files: 267
PDF files: 297


TOtal files: 564 plus 1 files as image.


In [5]:
# function to validate if the file exist
def file_exist(name):
    for file in all_files:
        if name == file.split(".")[0]:
            return True
    return False


In [6]:
# Create a new column called "File Exists" that indicates if the file exists in the directory
df_exist["File Exists"] = df_exist["filename"].apply(file_exist)

# print only the rows where the file does not exist
df_exist[~df_exist["File Exists"]]


,URL,Section,Subsection,Title,filename,File Exists
0,Account%20Access%20Error%20Support%20Knowledge...,['Software Systems'],['Technical Support'],['Account Access Error Support Knowledge Artic...,17b1c40e,False
294,https://missionaries.prod.byu-pathway.psdops.c...,['Shepherding / Ministering'],['Missing'],['Resources for New and Continuing Students'],269ed2e7,False
374,https://office365lds.sharepoint.com/sites/BYUP...,['Gatherings'],['Other Gathering Resources'],['Improving Your Role in the Gathering'],4208553,False
373,https://office365lds.sharepoint.com/sites/BYU-...,['Missionary Software & Uses'],['Troubleshooting'],['Check to See if a BYU-Pathway Worldwide Syst...,45762537,False
4,https://churchofjesuschrist.sharepoint.com/sit...,['English Connect 1&2'],['EnglishConnect Global Launch'],['EnglishConnect Global Launch'],563343d4,False
3,https://churchofjesuschrist.sharepoint.com/sit...,['Gatherings'],['PC102'],['Forming Team Project Groups'],72713f0a,False
372,https://office365lds.sharepoint.com/sites/BYU-...,['Area Coordination (For ACMs Only)'],['ACC Council'],['ACM Council Agendas'],78008ef2,False
562,https://degreeapplication.byupathway.edu/,NaN,NaN,NaN,d26452c9,False


562.- Degree application broken

294.- PNG image

0.- is not a link

All the other links are ignored because they ask to login.


In [7]:
# print len of existing files
print("Existing files:", len(df_exist[df_exist["File Exists"]]))


Existing files: 564


In [8]:
# Create an Array with the names of the files that do not exist
files_not_exist = df_exist[~df_exist["File Exists"]]["filename"].values


print(len(files_not_exist), "files do not exist")


8 files do not exist


132 files in the list doesn't exist in the download folder.


# 03 Verify if every file in quotes has been downloaded


In [19]:
filename = "../data/temporary/Index_quotes_26.csv"

df = pd.read_csv(
    filename,
    header=1,
)
df.index = range(1, len(df) + 1)

df.dropna(subset=["Link to Ideal Answer"], inplace=True)

# drop columns "Quotes", "Quote 2" and "Quote 3"
df.drop(columns=["Quotes", "Quote 2", "Quote 3"], inplace=True)

df_no_links = df[df["Link to Ideal Answer"].str.contains("http") == False]

# make a drop of the rows that have no link to ideal answer
df.drop(df_no_links.index, inplace=True)
df.head()


,Questions,Ideal Answer,Link to Ideal Answer,Link 2,Link 3
2,How do I know if a student has a scholarship?,Missionaries can see if a student has a schola...,https://missionaries.prod.byu-pathway.psdops.c...,NaN,NaN
3,How do I know if a student is registered for a...,There is not a way for Missionaries to verify ...,https://missionaries.prod.byu-pathway.psdops.c...,NaN,NaN
4,How do I know if a student is a returned missi...,Missionaries can see if a student is a returne...,https://missionaries.prod.byu-pathway.psdops.c...,NaN,NaN
5,How do I know if student is member of the church?,Missionaries can see the Church membership sta...,https://missionaries.prod.byu-pathway.psdops.c...,NaN,NaN
6,What information should I track for each student?,The most important things Missionaries should ...,https://missionaries.prod.byu-pathway.psdops.c...,NaN,NaN


In [20]:
# Función para limpiar y extraer los links
def extract_links(text):
    # si el valor es NaN, retornar una lista vacía
    if pd.isnull(text):
        return []

    # Dividir el texto por los saltos de línea y filtrar solo los enlaces
    links = [
        line.strip() for line in text.split("\n") if line.strip().startswith("https")
    ]
    return links


# Recopila los link de "Link to Ideal Answer", "Link 2" y "Link 3" en un solo campo
def merge_links(row):
    links = row["Link to Ideal Answer"] + row["Link 2"] + row["Link 3"]
    return links


# because some questions has multiple links, we will split them into multiple rows with its respective question
def split_links(row):
    links = row["Links"]
    questions = row["Questions"]
    rows = []
    for link in links:
        rows.append([questions, link])
    return rows


In [21]:
# some of the rows has multiple links, we will convert them to a list
df["Link to Ideal Answer"] = df["Link to Ideal Answer"].apply(extract_links)
df["Link 2"] = df["Link 2"].apply(extract_links)
df["Link 3"] = df["Link 3"].apply(extract_links)

df["Links"] = df.apply(merge_links, axis=1)

list_of_links = df[["Questions", "Links"]]

# Apply the function to the dataframe and convert the result to a dataframe
df_links = list_of_links.apply(split_links, axis=1)
df_links = pd.DataFrame(df_links.sum(), columns=["Questions", "Links"])
# drop duplicated Links
df_links.drop_duplicates(inplace=True)
df_links.head()


,Questions,Links
0,How do I know if a student has a scholarship?,https://missionaries.prod.byu-pathway.psdops.c...
1,How do I know if a student is registered for a...,https://missionaries.prod.byu-pathway.psdops.c...
2,How do I know if a student is a returned missi...,https://missionaries.prod.byu-pathway.psdops.c...
3,How do I know if student is member of the church?,https://missionaries.prod.byu-pathway.psdops.c...
4,What information should I track for each student?,https://missionaries.prod.byu-pathway.psdops.c...


Now with the links, verify if every of this resources has been downloaded.


In [22]:
# compare the Links from df_links to URL of df_exist, also File Exist must be True


def validate_if_exist(row):
    link = row["Links"]
    # verify if it exist in df_exist dataframe and its colum "File Exists" is True
    exist = df_exist[(df_exist["URL"].isin([link])) & (df_exist["File Exists"] == True)]
    return len(exist) > 0


df_links["File Exists"] = df_links.apply(validate_if_exist, axis=1)


In [23]:
# show all false values
df_some = df_links[~df_links["File Exists"]]

df_some

# show all true values
# df_links[df_links["File Exists"]]


,Questions,Links,File Exists
5,What to do If a student has already taken this...,https://pathway-missionary.powerappsportals.co...,False
16,How do students retake 102 without retaking 101?,https://pathway-missionary.powerappsportals.co...,False
18,Does a student need to retake PC 103 to retake...,https://pathway-missionary.powerappsportals.co...,False
54,What is the link to access the Service Mission...,https://missionaries.prod.byu-pathway.psdops.c...,False
75,How to avoid WhatsApp cancelling my account fo...,https://faq.whatsapp.com/361005896189245?helpr...,False
80,How do I add students to a WhatsApp group?,https://faq.whatsapp.com/361005896189245?helpr...,False
134,How do students access the BYUI application?,https://degreeapplication.byupathway.edu/,False


About the files above:

5, 16 and 18 The file exist, is 5ff7e07b.html

54 is an index webpage. it will not be downloaded

75 and 80 are the same, it is downloaded as b0590386.html

134 is link broken


# 04. Validate if Files were transformed to Markdown


In [13]:
import os


In [14]:
# origin

html_files = os.listdir("../data/data_test_sep_27/crawl/html")
pdf_files = os.listdir("../data/data_test_sep_27/crawl/pdf")

# remove the extension of the files
html_files = [file.split(".")[0] for file in html_files]
pdf_files = [file.split(".")[0] for file in pdf_files]


In [15]:
html_files_out = os.listdir("../data/data_test_sep_27/out/from_html")
pdf_files_out = os.listdir("../data/data_test_sep_27/out/from_pdf")

# drop txt files
html_files_out = [file for file in html_files_out if file.endswith(".md")]
pdf_files_out = [file for file in pdf_files_out if file.endswith(".md")]

# remove the extension of the files
html_files_out = [file.split(".")[0] for file in html_files_out]
pdf_files_out = [file.split(".")[0] for file in pdf_files_out]


In [16]:
# verify if the file exists in the directory "out_sep_12/from_html" or "out_sep_12/from_pdf", create a list of the files that do not exist
# rememeber to avoid the extension of the file
dont_exist_html = []
dont_exist_pdf = []

for file in html_files:
    if file.split(".")[0] not in html_files_out:
        dont_exist_html.append(file)

for file in pdf_files:
    if file.split(".")[0] not in pdf_files_out:
        dont_exist_pdf.append(file)


print("HTML files that do not exist:", len(dont_exist_html))
print("PDF files that do not exist:", len(dont_exist_pdf))


HTML files that do not exist: 0
PDF files that do not exist: 1


In [17]:
print("len of dont_exist_pdf:", len(dont_exist_pdf))
print("\n")
print(dont_exist_pdf)


len of dont_exist_pdf: 1


['95e946eb']


WARNING! This file, is a PDF file, and for a strange reason, when is converted to txt, it has many weird characters that prevents to run the parser.


# 05. Verify empty Files

---


In [25]:
from utils.tools import get_files


In [26]:
def is_empty_content(content):
    content = content.replace("\n", "").replace(" ", "")
    return not content


In [27]:
all_files = get_files("../data/data_test_sep_27/out")


In [28]:
for file in all_files:
    with open(file, "r") as f:
        content = f.read()
        # remove the metadata that is at the beginning of the file and between the "---"
        content = re.sub(r"---[\s\S]*?---", "", content)
        if is_empty_content(content):
            print(file)
